In [5]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

1. Data Loader

In [2]:
class WeatherDataset(Dataset):
    def __init__(self, folder_path):
        self.labels = {'Sunny': 0, 'Snowy': 1, 'Cloudy': 2, 'Rainy': 3, 'Foggy': 4}
        self.data = []
        for file in os.listdir(folder_path):
            label_name = file.split('.')[0]  # 假设文件名的格式是'Cloudy1.jpg'
            for label in self.labels:
                if label in label_name:
                    self.data.append((os.path.join(folder_path, file), self.labels[label]))
                    break

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)
    
    def one_hot_encode(self, label, num_classes=5):
        return torch.eye(num_classes)[label]

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        label = self.one_hot_encode(label)  # 将标签转换为one-hot编码
        return image, label


dataset = WeatherDataset(folder_path='../Data_Q2/train_data')
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [6]:
model = models.vgg16()
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 5)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(50): 
    epoch_loss = 0.0
    with tqdm(enumerate(dataloader, 0), total=len(dataloader), desc=f'Epoch {epoch+1}/{50}', unit='batch') as tepoch:
        for i, data in tepoch:
            inputs, labels = data
            labels = labels.float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            tepoch.set_postfix(loss=epoch_loss/(i+1))


Epoch 50/50: 100%|██████████| 63/63 [00:46<00:00,  1.37batch/s, loss=0.02]   


In [12]:
correct = 0
total = 0

import time
with torch.no_grad():
    for data in dataloader:
        images, labels = data
        labels = labels.float()
        outputs = model(images)
        predicted = outputs.sigmoid() > 0.5
        total += labels.numel()
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 250 images: {accuracy:.2f} %')

tensor([[0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])
tensor([[ -8.6623,  -8.4122,  -5.7530,  -7.2724,   5.5631],
        [ -5.0857, -13.7917,  -5.7141,   2.6622, -10.9708],
        [-13.5923,  13.1232, -13.4424, -13.2566, -14.8945],
        [ 16.2681, -15.8660, -10.3496, -17.8510, -16.7297]])
tensor([[False, False, False, False,  True],
        [False, False, False,  True, False],
        [False,  True, False, False, False],
        [ True, False, False, False, False]])
tensor([[0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])
tensor([[-15.2346,   7.3587, -11.2746,  -8.7636, -17.5033],
        [-15.6657, -17.0003, -14.3399, -11.0225,  13.5827],
        [  9.3751, -13.8857, -13.4379,  -9.3887, -23.0065],
        [-10.2229, -13.5265,  -5.2158, -11.7182,   2.8209]])
tensor([[False,  True, False, False, False],
        [False, False, False, False,  True],
     

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import CategoricalCrossentropy

In [3]:
def build_vgg16(input_shape=(224, 224, 3), num_classes=5):
    # Define the input
    input_tensor = Input(shape=input_shape)

    # First block
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_tensor)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Second block
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Third block
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Fourth block
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Fifth block
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Flatten and Dense layers
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=x)
    return model

In [4]:
# Dataset preparation
train_folder_path = '../Data_Q2/train_data'
batch_size = 4

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_folder_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Build and compile the model
model = build_vgg16()
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), 
              loss=CategoricalCrossentropy(), 
              metrics=['accuracy'])

# Training
model.fit(train_generator, epochs=50)

# Evaluation
accuracy = model.evaluate(train_generator)
print(f'Accuracy: {accuracy[1] * 100:.2f} %')

Found 0 images belonging to 0 classes.


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# 数据目录
data_dir = '../Data_Q2/train_data'

# 设置图像大小和批次大小
image_size = (224, 224)
batch_size = 32

# 创建ImageDataGenerator实例来加载和预处理图像
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# 创建训练集数据生成器
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

Found 0 images belonging to 0 classes.


{}

In [5]:
def build_vgg16(input_shape=(224, 224, 3), num_classes=5):
    # Define the input
    input_tensor = Input(shape=input_shape)

    # First block
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_tensor)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Second block
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Third block
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Fourth block
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Fifth block
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Flatten and Dense layers
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=x)
    return model

In [ ]:
model = build_vgg16()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
epochs = 50
history = model.fit(train_generator, epochs=epochs)


In [ ]:
# 在训练集上评估模型
accuracy = model.evaluate(train_generator)[1]
print(f"训练集上的准确率: {accuracy * 100:.2f}%")
